## Navigating crystal space

In [ ]:
# Installation of libraries
%pip install matminer --quiet
%pip install smact==3.2.0 --quiet
%pip install plotly --quiet
%pip install multiprocess --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import of modules
import pandas as pd  # Data manipulation with DataFrames
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Plotting
import plotly.express as px  # More plotting
import pprint  # Pretty print data structures

## ABX<sub>3</sub> perovskite structure

To represent a crystal structure in Python, the functionality for storing and representing crystal structures in `pymatgen` can be used. 

`Structure` can be used to represent the unit cell of a crystal.  This is the simplest repeat unit of a crystal that is defined by three unit cell lengths (**a**,**b**,**c**), three angles ($\alpha,\beta,\gamma$), and the fractional coordinates of the atoms that it contains.

Below, a model of a cubic perovskite will be created from its spacegroup, Pm3̅m (No. 221). For a cubic unit cell **a** = **b** = **c** and $\alpha = \beta = \gamma = 90 ^{\circ}$, only the length of **a** is needed to define.

A structure for CsPbI<sub>3</sub> will be created from a spacegroup using the class method `Structure.from_spacegroup()`. This requires the space group, lattice parameters, atom types, and coordinates.

In [3]:
# Create a structure object for cubic CsPbI3 using pymatgen
from pymatgen.core import Structure, Lattice  # Structure modules

CsPbI3 = Structure.from_spacegroup(
'Pm-3m', # Spacegroup for a cubic perovskite
Lattice.cubic(6.41), # Cubic spacing of 6.41 Å
['Pb2+', 'Cs+', 'I-'], # Unique species of the ABX3 compound
[[0.,0.,0.],[0.5,0.5,0.5],[0.,0.,0.5]] # Fractional atomic coordinates of each site
)

# Print the structure details
print(CsPbI3)

Full Formula (Cs1 Pb1 I3)
Reduced Formula: CsPbI3
abc   :   6.410000   6.410000   6.410000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Pb2+  0    0    0
  1  Cs+   0.5  0.5  0.5
  2  I-    0    0.5  0
  3  I-    0.5  0    0
  4  I-    0    0    0.5


Note that it has used the symmetry from the space group information to create a primitive unit cell with 5 atoms. Within `pymatgen`, crystal structures can be exported to a variety of file formats using the `to` method of the `Structure` class. Some of these include the Crystallographic Information Format (cif) and POSCAR (the standard file format for the density functional theory code [VASP](https://www.vasp.at/wiki/index.php/Category:Theory)). A complete list of file formats can be found in [the docs](https://pymatgen.org/pymatgen.core.structure.html#pymatgen.core.structure.IStructure.to) for the `Structure` class.

Note, the `filename` argument of the `to` method needs to be specified to save the file to disk. Only supplying the `fmt` argument will result in a string being returned.

In [4]:
# Print the file to screen
print('\nThe CIF format for CsPbI3 is shown below:\n')
print(CsPbI3.to(fmt='cif'))

# Export the structure to a CIF
# CsPbI3.to(filename='CsPbI3.cif')

# Download the CIF from Google Colab
# from google.colab import files
# files.download('CsPbI3.cif')


The CIF format for CsPbI3 is shown below:

# generated using pymatgen
data_CsPbI3
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   6.41000000
_cell_length_b   6.41000000
_cell_length_c   6.41000000
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   CsPbI3
_chemical_formula_sum   'Cs1 Pb1 I3'
_cell_volume   263.37472100
_cell_formula_units_Z   1
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_type_symbol
 _atom_type_oxidation_number
  Cs+  1.0
  Pb2+  2.0
  I-  -1.0
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Pb2+  Pb0  1  0.00000000  0.00000000  0.00000000  1
  Cs+  Cs1  1  0.50000000  0.50000000  0.50000000  1
  I-  I2  1  0.00000000  0.50000000  0.00000000  1
  I-  I3  1  0.50000000  0.00000000  0.00000000  1

There are many useful functions for analysing these structures. Due to using Python, this analysis can be extended to large databases of thousands to millions of entries. The following distances and two methods for assigning coordination numbers (detailed [here](https://pubs.acs.org/doi/10.1021/acs.inorgchem.0c02996)) will be checked.

In [5]:
# Distance between the first and second atoms
distance = CsPbI3.get_distance(0, 1)
print(f"Distance between Pb and Cs: {distance:.2f} Å")

# Coordination environment of Pb using Voronoi tessellation
from pymatgen.analysis.local_env import VoronoiNN
vnn = VoronoiNN()
coord_env = vnn.get_nn(CsPbI3, 0)
print(f"\nCoordination number of Pb (VoronoiNN): {len(coord_env)}")

# Coordination environment of Pb using Crystal Near-Neighbor Algorithm
from pymatgen.analysis.local_env import CrystalNN
crystal_nn = CrystalNN()
coord_env_crystal = crystal_nn.get_nn(CsPbI3, 0)
print(f"\nCoordination number of Pb (CrystalNN): {len(coord_env_crystal)}\n")
for site in coord_env_crystal:
    print(f"Distance to {site.species_string}: {CsPbI3.get_distance(0, site.index):.2f} Å")

# Assign oxidation states
from pymatgen.analysis.bond_valence import BVAnalyzer
analyzer = BVAnalyzer()
valences = analyzer.get_valences(CsPbI3)
print(f"\nAssigned oxidation states:")
for site, valence in zip(CsPbI3.sites, valences):
    print(f"{site.species_string}: {valence:+.1f}")

Distance between Pb and Cs: 5.55 Å

Coordination number of Pb (VoronoiNN): 6

Coordination number of Pb (CrystalNN): 6

Distance to I-: 3.21 Å
Distance to I-: 3.21 Å
Distance to I-: 3.21 Å
Distance to I-: 3.21 Å
Distance to I-: 3.21 Å
Distance to I-: 3.21 Å

Assigned oxidation states:
Pb2+: +2.0
Cs+: +1.0
I-: -1.0
I-: -1.0
I-: -1.0


In [6]:
from matminer.featurizers.structure import SineCoulombMatrix

# Sine matrix representation
scm = SineCoulombMatrix(flatten=False)
scm.fit([CsPbI3])
scm_fea = scm.featurize(CsPbI3)
np.set_printoptions(precision=2, suppress=True)
print("\nSine Coulomb matrix features:\n", np.array(scm_fea))


Sine Coulomb matrix features:
 [[[19594.01   214.96   358.78   358.78   358.78]
  [  214.96  7513.47   170.16   170.16   170.16]
  [  358.78   170.16  6874.36   163.97   163.97]
  [  358.78   170.16   163.97  6874.36   163.97]
  [  358.78   170.16   163.97   163.97  6874.36]]]


The output is a 5x5 matrix because there are 5 atoms in the unit cell of CsPbI<sub>3</sub>. Each element M<sub>ij</sub> in the matrix represents the interaction between atoms i and j.

- The **diagonal elements** (i.e., M<sub>ii</sub>) represent the interaction of each atom with itself, which depends on the atomic number Z<sub>i</sub> and is calculated as 0.5Z<sub>i</sub><sup>2.4</sup>.
- The **off-diagonal elements** (i.e., M<sub>ij</sub> for i ≠ j) are inspired by the Coulomb interaction between different atoms i and j, scaled by their atomic numbers Z<sub>i</sub> and Z<sub>j</sub>, and the distance between them in the periodic structure.

This matrix encodes information from both the positions and types of atoms, providing a feature vector. Let's see what happens when the chemistry changes.

In [7]:
from pymatgen.core import Species

# Substitute Pb with Sn
CsSnI3 = CsPbI3.copy()
CsSnI3.replace_species({Species("Pb2+"): Species("Sn2+")})

# Sine matrix representation for CsSnI3
scm = SineCoulombMatrix(flatten=False)
scm.fit([CsSnI3])
scm_fea_sn = scm.featurize(CsSnI3)

np.set_printoptions(precision=2, suppress=True)
print("\nOriginal Coulomb matrix features:\n", np.array(scm_fea))
print("\nSine Coulomb matrix features with Sn replacing Pb:\n", np.array(scm_fea_sn))


Original Coulomb matrix features:
 [[[19594.01   214.96   358.78   358.78   358.78]
  [  214.96  7513.47   170.16   170.16   170.16]
  [  358.78   170.16  6874.36   163.97   163.97]
  [  358.78   170.16   163.97  6874.36   163.97]
  [  358.78   170.16   163.97   163.97  6874.36]]]

Sine Coulomb matrix features with Sn replacing Pb:
 [[[5977.2   131.07  218.77  218.77  218.77]
  [ 131.07 7513.47  170.16  170.16  170.16]
  [ 218.77  170.16 6874.36  163.97  163.97]
  [ 218.77  170.16  163.97 6874.36  163.97]
  [ 218.77  170.16  163.97  163.97 6874.36]]]


While useful, this is not the state-of-the-art approach. One reason is that only pairwise information is included, e.g. two structures with different angles between the component atoms may have the same numerical representation. The [DScribe tutorials](https://singroup.github.io/dscribe/latest/tutorials/tutorials.html) contain a description of more advanced representations such as Smooth Overlap of Atomic Positions (SOAP) that includes both 2-body and 3-body descriptors of structure.

## Possible Perovskites

An ionic approximation for CsPbI<sub>3</sub>  can be made and consider that the crystal is formed of Cs<sup>+</sup>, Pb<sup>2+</sup>, and I<sup>-</sup> ions. The crystal is charge neutral overall as the sum of cations (3<sup>+</sup>) is balanced by the sum of anions (3<sup>-</sup>). It is worth remembering that bonding in real solids is more complicated than a simple classification of "ionic", "covalent" or "metallic" (e.g. see this [perspective](https://www.nature.com/articles/s41563-018-0165-7)).

Let's now screen a large space of possible Cs-Pb-I compounds by making use of our materials informatics code [smact](https://github.com/WMD-group/SMACT). This combinatorial chemical space will be navigated by applying charge balance filters to possible ABX₃ compounds.

In [9]:
import smact
from smact.screening import smact_filter
import pprint

# Define a list of three elements to screen for A-B-X compounds
elements = ["Cs", "Pb", "I"]

# Convert element symbols into smact.Element objects
element_objects = [smact.Element(e) for e in elements]

# Apply smact_filter to find valid compositions within a stoichiometry threshold
allowed_combinations = smact_filter(element_objects, threshold=8)

# Display the allowed combinations in a readable format
print(f"Number of allowed combinations: --> {len(allowed_combinations)} <--")
pprint.pprint("(elements), (charges), (stoichiometry)")
pprint.pprint(allowed_combinations)

Number of allowed combinations: --> 45 <--
'(elements), (charges), (stoichiometry)'
[(('Cs', 'Pb', 'I'), (1, -4, -1), (5, 1, 1)),
 (('Cs', 'Pb', 'I'), (1, -4, -1), (6, 1, 2)),
 (('Cs', 'Pb', 'I'), (1, -4, -1), (7, 1, 3)),
 (('Cs', 'Pb', 'I'), (1, -4, -1), (8, 1, 4)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (2, 1, 1)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (3, 1, 2)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (3, 2, 1)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (4, 1, 3)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (4, 3, 1)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (5, 1, 4)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (5, 2, 3)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (5, 3, 2)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (5, 4, 1)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (6, 1, 5)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (6, 5, 1)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (7, 1, 6)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (7, 2, 5)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (7, 3, 4)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (7, 4, 3)),
 (('Cs', 'Pb', 'I'), (1, -1, -1), (7, 5, 2

In [ ]:
import itertools
import multiprocessing
from smact import Element, element_dictionary, ordered_elements
from smact.screening import smact_filter

# Load all elements and then limit to Bi
all_elements = element_dictionary()
elements = [all_elements[symbol] for symbol in ordered_elements(1, 83)]

# Generate all AB combinations + add oxygen (O)
metal_pairs = itertools.combinations(elements, 2)
ternary_systems = [[*pair, Element("O")] for pair in metal_pairs]
print(f"Raw element combinations (ABO): --> {len(ternary_systems)} <--")

# Apply chemical filters using multiprocessing
def run_filter(args):
    return smact_filter(*args)

if __name__ == "__main__":
    with multiprocessing.Pool(processes=4) as pool:
        result = pool.starmap(run_filter, ternary_systems)

# Flatten the list
flat_list = [composition for sublist in result if sublist is not None for composition in sublist]
print(f"Allowed stoichiometries: --> {len(flat_list)} <--")

# Print some valid compositions
print("  elements, oxidation states, stoichiometries")
for entry in flat_list[-10:]:
    print(entry)

Raw element combinations (ABO): --> 3403 <--


In [ ]:
import itertools
import multiprocess as mp
from smact import Element, element_dictionary, ordered_elements
from smact.screening import smact_filter

all_elements = element_dictionary()
elements = [all_elements[symbol] for symbol in ordered_elements(1, 83)]

O = Element("O")
ternary_systems = [[a, b, O] for a, b in itertools.combinations(elements, 2)]
print(f"Raw element combinations (ABO): --> {len(ternary_systems)} <--")

def safe_smact_filter(system):
    """Avoid TypeErrors from smact_filter in multiprocessing."""
    try:
        return smact_filter(system)
    except TypeError:
        return []

with mp.Pool(processes=4) as pool:
    result = pool.map(safe_smact_filter, ternary_systems)

flat_list = [comp for sub in result if sub for comp in sub]
print(f"Allowed stoichiometries: --> {len(flat_list)} <--")

In [ ]:
# Filter for (1,1,3) stoichiometries with oxide anion
filtered_compositions = [
    comp for comp in flat_list
    if comp[2] == (1, 1, 3)
    and 'O' in comp[0]
    and comp[1][comp[0].index('O')] == -2  # Oxide
    and comp[1][0] > 0  # First element must have a positive charge
    and comp[1][1] > 0  # Second element must have a positive charge
]

print(f"Filtered (1,1,3) stoichiometries with O(-2): --> {len(filtered_compositions)} <--")

# Print a few filtered compositions
print("  elements, oxidation states, stoichiometries")
for entry in filtered_compositions[-10:]:
    print(entry)

In [ ]:
# List of elements to include
elements = sorted(set([el for comp in filtered_compositions for el in comp[0]]))

# DataFrame to hold the allowed combinations
matrix = pd.DataFrame(0, index=elements, columns=elements)

# Fill the matrix with allowed combinations
for comp in filtered_compositions:
    A, B, O = comp[0]
    matrix.at[A, B] = 1
    matrix.at[B, A] = 1

# Create the plot
fig = px.imshow(
    matrix.values,
    labels=dict(x="Element A", y="Element B", color="Allowed"),
    x=matrix.columns,
    y=matrix.index,
    color_continuous_scale="Plasma",
)

# Update layout for smaller font size
fig.update_layout(
    xaxis=dict(tickfont=dict(size=10)),
    yaxis=dict(tickfont=dict(size=10)),
    title="Charge allowed ABO<sub>3</sub> combinations",
    title_x=0.5,
)

fig.show()